In [11]:
import os
from PyPDF2 import PdfReader
import re


# Get the database root directory
try:
    root_dir = os.path.join(os.path.dirname(__file__), 'database', 'queplan_insurance')
except:
    print("This should not be in a Jupyter Notebook.")
    root_dir = os.path.join(os.path.dirname('.'), 'database', 'queplan_insurance')

# Get the list of PDF files in the directory
pdf_files = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith('.pdf')]

# Dictionary to store the results
patron = "cobertura"

# Function to count the pages of a PDF
def count_pages(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        return len(reader.pages)

# Function to count the titles of a PDF
def count_titles(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return len(re.findall(r'\b(cobertura)\b', text))

# Process each PDF file
results = {}
for pdf_file in pdf_files:
    pdf_name = pdf_file.split('.')[0]
    pages = count_pages(pdf_file)
    titles = count_titles(pdf_file)
    results[pdf_name] = {'pages': pages, 'titles': titles}

# Print results
for pdf, info in results.items():
    print(f"{pdf}: Pages: {info['pages']}, Titles: {info['titles']}")


This should not be in a Jupyter Notebook.
database\queplan_insurance\POL120190177: Pages: 14, Titles: 17
database\queplan_insurance\POL320130223: Pages: 47, Titles: 31
database\queplan_insurance\POL320150503: Pages: 26, Titles: 17
database\queplan_insurance\POL320180100: Pages: 25, Titles: 19
database\queplan_insurance\POL320190074: Pages: 72, Titles: 68
database\queplan_insurance\POL320200071: Pages: 25, Titles: 34
database\queplan_insurance\POL320200214: Pages: 42, Titles: 44
database\queplan_insurance\POL320210063: Pages: 6, Titles: 11
database\queplan_insurance\POL320210210: Pages: 10, Titles: 11


In [7]:
# root_dir = os.path.join(os.path.dirname(__file__), 'database', 'queplan_insurance')
root_dir = os.path.join(os.path.dirname('.'), 'database', 'queplan_insurance')
os.listdir(root_dir)

['POL120190177.pdf',
 'POL320130223.pdf',
 'POL320150503.pdf',
 'POL320180100.pdf',
 'POL320190074.pdf',
 'POL320200071.pdf',
 'POL320200214.pdf',
 'POL320210063.pdf',
 'POL320210210.pdf']